In [2]:
from ultralytics import YOLO

# Carrega o modelo treinado para a rede neural YOLO
model = YOLO('./models/license_plate_detector.pt')  # load a pretrained model (recommended for training)

# Usa o modelo para encontrar a placa veicular no video.
results = model(source="video/fiat/fiat1.mp4",#diretório de origem para imagens ou vídeos 
                      conf=0.85,#limite de confiança do objeto para detecção 
                      show=False, #mostre os resultados se possível 
                      save=True, #salvar imagens com resultados
                      save_crop=True, #salve imagens recortadas com resultados
                      max_det=5, #número máximo de detecções por imagem
                      visualize=False #visualizar recursos do modelo
                      )

# source= 0 Habilita webcam

c:\Users\rafae\anaconda3\Lib\site-packages\torch\cuda\__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Ma

In [ ]:
#TRACK ID
from ultralytics import YOLO

# load models
model = YOLO('./models/license_plate_detector.pt')

#run inference on the source
results = model.track(source="video/faculdade/Trim5.mp4",#diretório de origem para imagens ou vídeos 
                      conf=0.85,#limite de confiança do objeto para detecção 
                      show=False, #mostre os resultados se possível 
                      tracker="bytetrack.yaml", 
                      save=False, #salvar imagens com resultados
                      save_crop=True, #salve imagens recortadas com resultados
                      max_det=2, #número máximo de detecções por imagem
                      visualize=False, #visualizar recursos do modelo
                      persist=True # Se os rastreadores devem persistir se eles já existirem (testar)
                      ) 

In [ ]:
#Entender funcionamento do codigo para funcionar em conjunto com OCR - Tesseract
import ast
import cv2 as cv
import easyocr
import numpy as np
import pandas as pd
import string
from ultralytics import YOLO

# Load models

# regular pre-trained yolov8 model for car recognition
coco_model = YOLO('yolov8s.pt')
# coco_model = YOLO('yolov8n.pt')

# yolov8 model trained to detect number plates
model = YOLO('./models/license_plate_detector.pt')

# read video by index
video = cv.VideoCapture('./video/fiat/fiat11.mp4')

ret = True
frame_number = -1
vehicles = [2]

while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret:
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.85:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    #print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    # debugging check if bbox lines up with detected vehicles (should be identical to save_crops() above
                    #cv.imwrite(str(track_id) + '.jpg', roi)
                    
                    # license plate detector for region of interest
                    license_plates = model(roi)[0]
                   
                    # check every bounding box for a license plate
                    for license_plate in license_plates.boxes.data.tolist():
                     plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                     if plate_score > 0.80:
                         #print("funciona")
                         license_plates.save_crop('outputs')# Variavel "license_plates" para jogar no pre-processamento e fazer OCR 
                         #print(license_plate, 'track_id: ' + str(bbox[4]))
                         plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                         cv.imwrite(str(track_id) + '.jpg', plate)
                                             
video.release()